# Making Filter Protitypes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import constants
import sympy
import random
import pandas as pd
import itertools


num_of_sections = 3
num_of_combinations = 2**num_of_sections
l = 100
f = np.linspace(1e3, 10e9, l)

L0 = 1e-7

C1 = 3e-9
C2 = 3e-10

# Generate all possible combinations of an 8-bit input
input_combinations = list(itertools.product([0, 1], repeat=8))

# Convert the combinations to a NumPy array
input_matrix = np.array(input_combinations)

C = input_matrix*C1 + (1-input_matrix)*C2

Rs = 50
RL = 50


def update_Z_load(f, L0, C, Z_load):
    return Z_load/(f*Z_load*C+1) + 2*1j*f*L0

def find_ABCD(f, L0, C, Z_load, l):
    a = np.array(l, dtype=np.complex256)
    b = np.array(l, dtype=np.complex256)
    c = np.array(l, dtype=np.complex256)
    d = np.array(l, dtype=np.complex256)

    a = -2*f**2*C*L0 + 1
    b = -f**2 *C*Z_load*L0 + 1j*f*L0
    c = (1j*f*C)/(-2*f**2 *C*L0 + 1)
    d = 1j*C*Z_load + 1
    return a, b, c, d

def find_S_matrix(a, b, c, d, f, L0, C, Z_load, l):
    S11 = np.array(l, dtype=np.complex256)
    S12 = np.array(l, dtype=np.complex256)
    S21 = np.array(l, dtype=np.complex256)
    S22 = np.array(l, dtype=np.complex256)

    Z0 = np.sqrt(L0/C)
    S11 = (a + b/Z0 - c/Z0 + d)/(a + b/Z0 + c/Z0 + d)
    S12 = 2 * (a*d - b*c) / (a + b/Z0 + c*Z0 + d)
    S21 = 2 / (a + b/Z0 + c*Z0 + d)
    S22 = (-a + b/Z0 - c*Z0 + d) / (a + b/Z0 + c*Z0 + d)
    return S11, S12, S21, S22

# Plot in log-log scale
plt.figure()

for i in range(num_of_combinations):
    Z_load = RL*np.ones(l, dtype=np.complex256) # initialisation
    for j in range(num_of_sections):
        a, b, c, d = find_ABCD(f, L0, C[i][j], Z_load, l)
        S11_new, S12_new, S21_new, S22_new = find_S_matrix(a, b, c, d, f, L0, C[i][j], Z_load, l)

        if (j == 0):
            S11 = S11_new
            S21 = S21_new
            S12 = S12_new
            S22 = S22_new
        else:
            S11 = S11*S11_new + S12*S21_new
            S12 = S11*S12_new + S12*S22_new
            S21 = S21*S11_new + S22*S21_new
            S22 = S21*S12_new + S22*S22_new

        Z_load = update_Z_load(f, L0, C[i][j], Z_load)
    plt.semilogx(f, 20 * np.log10(np.abs(S12)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S12 (dB)')
plt.title('S12 vs Frequency')
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import constants
import sympy
import random
import pandas as pd
import itertools


num_of_sections = 1
num_of_combinations = 2**num_of_sections
l = 100
f = np.linspace(1, 10e9, l)

L0 = 1e-7

C1 = 3e-9
C2 = 3e-10

# Generate all possible combinations of an 8-bit input
input_combinations = list(itertools.product([0, 1], repeat=8))

# Convert the combinations to a NumPy array
input_matrix = np.array(input_combinations)

C = input_matrix*C1 + (1-input_matrix)*C2

Rs = 50
RL = 50


def update_Z_load(f, L0, C, Z_load):
    return Z_load/(f*Z_load*C+1) + 2*1j*f*L0

def find_ABCD(f, L0, C, Z_load, l):
    a = np.array(l, dtype=np.complex256)
    b = np.array(l, dtype=np.complex256)
    c = np.array(l, dtype=np.complex256)
    d = np.array(l, dtype=np.complex256)

    a = np.abs(-2*f**2*C*L0 + 1)
    b = np.abs(-f**2 *C*Z_load*L0 + 1j*f*L0)
    c = np.abs((1j*f*C)/(-2*f**2 *C*L0 + 1))
    d = np.abs(1j*C*Z_load + 1)
    return a, b, c, d

def find_S_matrix(a, b, c, d, f, L0, C, Z_load, l):
    S11 = np.array(l, dtype=np.complex256)
    S12 = np.array(l, dtype=np.complex256)
    S21 = np.array(l, dtype=np.complex256)
    S22 = np.array(l, dtype=np.complex256)

    Z0 = np.sqrt(L0/C)
    S11 = (a + b/Z0 - c/Z0 + d)/(a + b/Z0 + c/Z0 + d)
    S12 = 2 * (a*d - b*c) / (a + b/Z0 + c*Z0 + d)
    S21 = 2 / (a + b/Z0 + c*Z0 + d)
    S22 = (-a + b/Z0 - c*Z0 + d) / (a + b/Z0 + c*Z0 + d)
    return S11, S12, S21, S22

# Plot in log-log scale
plt.figure()

for i in range(num_of_combinations):
    Z_load = RL*np.ones(l, dtype=np.complex256) # initialisation
    for j in range(num_of_sections):
        a, b, c, d = find_ABCD(f, L0, C[i][j], Z_load, l)
        S11_new, S12_new, S21_new, S22_new = find_S_matrix(a, b, c, d, f, L0, C[i][j], Z_load, l)

        if (j == 0):
            S11 = S11_new
            S21 = S21_new
            S12 = S12_new
            S22 = S22_new
        else:
            S11 = S11*S11_new + S12*S21_new
            S12 = S11*S12_new + S12*S22_new
            S21 = S21*S11_new + S22*S21_new
            S22 = S21*S12_new + S22*S22_new

        Z_load = update_Z_load(f, L0, C[i][j], Z_load)
    plt.semilogx(f, 20 * np.log10(np.abs(S12)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S12 (dB)')
plt.title('S12 vs Frequency')
plt.grid()
plt.show()

In [ ]:
print(S12)

import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import constants
import sympy
import random
import pandas as pd
import itertools


num_of_sections = 8
num_of_combinations = 2**num_of_sections
l = 10000
f = np.linspace(1e3, 10e9, l)

L0 = 1e-7

C1 = 3e-9
C2 = 3e-10

# Generate all possible combinations of an 8-bit input
input_combinations = list(itertools.product([0, 1], repeat=8))

# Convert the combinations to a NumPy array
input_matrix = np.array(input_combinations)

C = input_matrix*C1 + (1-input_matrix)*C2

Rs = 50
RL = 50
Z_load = RL*np.ones(l, dtype=np.complex256) # initialisation


def update_Z_load(f, L0, C, Z_load):
    return Z_load/(f*Z_load*C+1) + 2*1j*f*L0

def find_ABCD(f, L0, C, Z_load, l):
    a = np.array(l, dtype=np.complex256)
    b = np.array(l, dtype=np.complex256)
    c = np.array(l, dtype=np.complex256)
    d = np.array(l, dtype=np.complex256)

    a = np.abs(-2*f**2*C*L0 + 1)
    b = np.abs(-f**2 *C*Z_load*L0 + 1j*f*L0)
    c = np.abs((1j*f*C)/(-2*f**2 *C*L0 + 1))
    d = np.abs(1j*C*Z_load + 1)
    return a, b, c, d

def find_S_matrix(a, b, c, d, f, L0, C, Z_load, l):
    S11 = np.array(l, dtype=np.complex256)
    S12 = np.array(l, dtype=np.complex256)
    S21 = np.array(l, dtype=np.complex256)
    S22 = np.array(l, dtype=np.complex256)

    Z0 = np.sqrt(L0/C)
    S11 = (a + b/Z0 - c/Z0 - d)/(a + b/Z0 + c/Z0 + d)
    S12 = 2 * (a*d - b*c) / (a + b/Z0 + c*Z0 + d)
    S21 = 2 / (a + b/Z0 + c*Z0 + d)
    S22 = (-a + b/Z0 - c*Z0 + d) / (a + b/Z0 + c*Z0 + d)
    return S11, S12, S21, S22

# Plot in log-log scale
plt.figure()

for i in range(2):
    Z_load = RL*np.ones(l, dtype=np.complex256) # initialisation
    for j in range(num_of_sections):
        a, b, c, d = find_ABCD(f, L0, C[i][j], Z_load, l)
        S11_new, S12_new, S21_new, S22_new = find_S_matrix(a, b, c, d, f, L0, C[i][j], Z_load, l)

        if (j == 0):
            S11 = S11_new
            S21 = S21_new
            S12 = S12_new
            S22 = S22_new
        else:
            S11 = S11*S11_new + S12*S21_new
            S12 = S11*S12_new + S12*S22_new
            S21 = S21*S11_new + S22*S21_new
            S22 = S21*S12_new + S22*S22_new

        Z_load = update_Z_load(f, L0, C[i][j], Z_load)
    plt.semilogx(f, 20 * np.log10(np.abs(S22)))  # Convert S12 to dB

plt.xlabel('Frequency (Hz)')
plt.ylabel('S12 (dB)')
plt.title('S12 vs Frequency')
plt.grid()
plt.show()

"""a, b, c, d = find_ABCD(f, L0, C1, Z_load, l)
S11, S12, S21, S22 = find_S_matrix(a, b, c, d, f, L0, C1, Z_load, l)

plt.figure()
plt.semilogx(f, 20 * np.log10(np.abs(S12)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S12 (dB)')
plt.title('S12 vs Frequency')
plt.grid()
#plt.show()

plt.figure()
plt.semilogx(f, 20 * np.log10(np.abs(S21)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S21 (dB)')
plt.title('S21 vs Frequency')
plt.grid()
#plt.show()

plt.figure()
plt.semilogx(f, 20 * np.log10(np.abs(S11)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S11 (dB)')
plt.title('S11 vs Frequency')
plt.grid()
#plt.show()

plt.figure()
plt.semilogx(f, 20 * np.log10(np.abs(S22)))  # Convert S12 to dB
    
plt.xlabel('Frequency (Hz)')
plt.ylabel('S22 (dB)')
plt.title('S22 vs Frequency')
plt.grid()
plt.show()"""